In [ ]:
import os, sys
import numpy as np
import torch
from astropy.table import Table
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

sys.path.append("../..")

from astroclip.env import format_with_env
from property_utils.models import few_shot, zero_shot
from property_utils.plotting import plot_scatter

ASTROCLIP_ROOT = "/workspace/astroclip/"

PROVABGS_ROOT = f"/workspace/astroclip/data/"
# SUPERVISED_ROOT = f"{ASTROCLIP_ROOT}/supervised/"
ASTROCLIP_ROOT = "/workspace/astroclip"
PRETRAINED_ROOT = f"{ASTROCLIP_ROOT}"
PROVABGS_ROOT = f"{ASTROCLIP_ROOT}/data/"
# Define models in embeddings
image_models = ["astrodino", "xcit_encoder","swin_encoder","swinv2_encoder", "vim_encoder"]
spectrum_models =  ["specformer", "astroclip_spectrum"]

# Set up the paths
train_path = os.path.join(PROVABGS_ROOT, "provabgs_train_ssl_embeddings.hdf5")
test_path = os.path.join(PROVABGS_ROOT, "provabgs_test_ssl_embeddings.hdf5")

# Get embeddings and PROVABGS table
train_provabgs = Table.read(train_path)
test_provabgs = Table.read(test_path)

# Get properties and scale
properties = ["Z_MW", "LOG_MSTAR", "TAGE_MW", "sSFR"]
y_train = np.stack([train_provabgs[prop].data.squeeze() for prop in properties]).T
y_test = np.stack([test_provabgs[prop].data.squeeze() for prop in properties]).T
scaler = {"mean": y_train.mean(axis=0), "std": y_train.std(axis=0)}
y_train = (y_train - scaler["mean"]) / scaler["std"]

print(
    "Size of training set:",
    len(train_provabgs),
    "\nSize of test set:",
    len(test_provabgs),
)



# Galaxy Property Prediction from Image Embeddings

In [4]:
# Get data
data = {}
for model in image_models:
    data[model] = {}
    X_train, X_test = (
        train_provabgs[model + "_embeddings"],
        test_provabgs[model + "_embeddings"],
    )
    X_train = X_train.reshape(X_train.shape[0],-1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    embedding_scaler = StandardScaler().fit(X_train)
    data[model]["train"] = embedding_scaler.transform(X_train)
    data[model]["test"] = embedding_scaler.transform(X_test)

In [5]:
# Perfrom knn and mlp
preds_knn, preds_mlp = {}, {}
for key in data.keys():
    print(f"Evaluating {key} model...")
    raw_preds_knn = zero_shot(data[key]["train"], y_train, data[key]["test"])
    raw_preds_mlp = few_shot(
        model, data[key]["train"], y_train, data[key]["test"]
    ).squeeze()
    preds_knn[key] = raw_preds_knn * scaler["std"] + scaler["mean"]
    preds_mlp[key] = raw_preds_mlp * scaler["std"] + scaler["mean"]

Evaluating vim_encoder model...


In [9]:
# Make a table of r^2 scores
knn_prop = {key: [] for key in preds_knn.keys()}
mlp_prop = {key: [] for key in preds_mlp.keys()}

for key in preds_knn.keys():
    for i, prop in enumerate(properties):
        knn_prop[key].append(r2_score(y_test[:, i], preds_knn[key][:, i]))
        knn_prop[key].append(mean_absolute_error(y_test[:, i], preds_knn[key][:, i]))
        knn_prop[key].append(mean_squared_error(y_test[:, i], preds_knn[key][:, i]))
        knn_prop[key].append(pearsonr(y_test[:, i], preds_knn[key][:, i])[0])
        knn_prop[key].append(spearmanr(y_test[:, i], preds_knn[key][:, i])[0])

        mlp_prop[key].append(r2_score(y_test[:, i], preds_mlp[key][:, i]))
        mlp_prop[key].append(mean_absolute_error(y_test[:, i], preds_knn[key][:, i]))
        mlp_prop[key].append(mean_squared_error(y_test[:, i], preds_knn[key][:, i]))
        mlp_prop[key].append(pearsonr(y_test[:, i], preds_knn[key][:, i])[0])
        mlp_prop[key].append(spearmanr(y_test[:, i], preds_knn[key][:, i])[0])
knn_prop["properties"] = properties
mlp_prop["properties"] = properties

In [10]:
Table(knn_prop)

vim_encoder,properties
float64,str9
0.024720143054751786,Z_MW
0.05513223285710489,LOG_MSTAR
-0.007044884187193734,TAGE_MW
-1.072077465896021,sSFR


In [11]:
Table(mlp_prop)

vim_encoder,properties
float64,str9
0.2985458852642845,Z_MW
0.6053510601438861,LOG_MSTAR
0.07179152455067772,TAGE_MW
0.16810028903837704,sSFR


In [ ]:
# Get predictions from supervised models
resnet_preds = torch.load(
    os.path.join(SUPERVISED_ROOT, "image/ResNet18/global_properties/test_pred.pt")
)
photometry_preds = torch.load(
    os.path.join(SUPERVISED_ROOT, "photometry/MLP/global_properties/test_pred.pt")
)

# Add predictions to dictionary
preds_supervised = {
    "resnet18": np.stack([resnet_preds[prop].squeeze() for prop in properties]).T,
    "photometry": np.stack([photometry_preds[prop].squeeze() for prop in properties]).T,
}

supervised_r2 = {key: [] for key in preds_supervised.keys()}
for key in preds_supervised.keys():
    for i, prop in enumerate(properties):
        supervised_r2[key].append(r2_score(y_test[:, i], preds_supervised[key][:, i]))

supervised_r2["properties"] = properties
Table(supervised_r2)

# Galaxy Property Prediction from Spectrum Embeddings

In [ ]:
# Get data
data = {}
for model in spectrum_models:
    data[model] = {}
    X_train, X_test = (
        train_provabgs[model + "_embeddings"],
        test_provabgs[model + "_embeddings"],
    )
    embedding_scaler = StandardScaler().fit(X_train)
    data[model]["train"] = embedding_scaler.transform(X_train)
    data[model]["test"] = embedding_scaler.transform(X_test)

In [ ]:
# Perfrom knn and mlp
preds_knn, preds_mlp = {}, {}
for key in data.keys():
    print(f"Evaluating {key} model...")
    raw_preds_knn = zero_shot(data[key]["train"], y_train, data[key]["test"])
    raw_preds_mlp = few_shot(
        model, data[key]["train"], y_train, data[key]["test"]
    ).squeeze()
    preds_knn[key] = raw_preds_knn * scaler["std"] + scaler["mean"]
    preds_mlp[key] = raw_preds_mlp * scaler["std"] + scaler["mean"]

In [ ]:
# Make a table of r^2 scores
knn_r2 = {key: [] for key in preds_knn.keys()}
mlp_r2 = {key: [] for key in preds_mlp.keys()}

for key in preds_knn.keys():
    for i, prop in enumerate(properties):
        knn_prop[key].append(r2_score(y_test[:, i], preds_knn[key][:, i]))
        knn_prop[key].append(mean_absolute_error(y_test[:, i], preds_knn[key][:, i]))
        knn_prop[key].append(mean_squared_error(y_test[:, i], preds_knn[key][:, i]))
        knn_prop[key].append(pearsonr(y_test[:, i], preds_knn[key][:, i])[0])
        knn_prop[key].append(spearmanr(y_test[:, i], preds_knn[key][:, i])[0])

        mlp_prop[key].append(r2_score(y_test[:, i], preds_mlp[key][:, i]))
        mlp_prop[key].append(mean_absolute_error(y_test[:, i], preds_knn[key][:, i]))
        mlp_prop[key].append(mean_squared_error(y_test[:, i], preds_knn[key][:, i]))
        mlp_prop[key].append(pearsonr(y_test[:, i], preds_knn[key][:, i])[0])
        mlp_prop[key].append(spearmanr(y_test[:, i], preds_knn[key][:, i])[0])
knn_prop["properties"] = properties
mlp_prop["properties"] = properties

In [ ]:
Table(knn_prop)

In [ ]:
Table(mlp_prop)

In [ ]:
# Get predictions from supervised models
spectrum_preds = torch.load(
    os.path.join(SUPERVISED_ROOT, "spectrum/Conv+Att/global_properties/test_pred.pt")
)

# Add predictions to dictionary
preds_supervised = {
    "conv+att": np.stack([spectrum_preds[prop].squeeze() for prop in properties]).T,
}

supervised_r2 = {key: [] for key in preds_supervised.keys()}
for key in preds_supervised.keys():
    for i, prop in enumerate(properties):
        supervised_r2[key].append(r2_score(y_test[:, i], preds_supervised[key][:, i]))

supervised_r2["properties"] = properties
Table(supervised_r2)